# Time-series API Emulator for MLB
This script emulates the time-series API of this competition to allow debugging in a local environment.

You can also emulate the public LB locally instead of submitting your code, since train_updated.csv also contains data for the public LB period.

This script is inspired by a great notebook by @its7171 at the Riiid competition.
https://www.kaggle.com/its7171/time-series-api-iter-test-emulator

## The emulator

In [1]:
import os
import warnings
from typing import Optional, Tuple

import pandas as pd


class Environment:
    def __init__(self,
                 data_dir: str,
                 eval_start_day: int,
                 eval_end_day: Optional[int],
                 use_updated: bool,
                 multiple_days_per_iter: bool):
        warnings.warn('this is mock module for mlb')

        postfix = '_updated' if use_updated else ''
        
        # recommend to replace this with pickle, feather etc to speedup preparing data
        df_train = pd.read_csv(os.path.join(data_dir, f'train{postfix}.csv'))

        players = pd.read_csv(os.path.join(data_dir, 'players.csv'))

        self.players = players[players['playerForTestSetAndFuturePreds'] == True]['playerId'].astype(str)
        if eval_end_day is not None:
            self.df_train = df_train.set_index('date').loc[eval_start_day:eval_end_day]
        else:
            self.df_train = df_train.set_index('date').loc[eval_start_day:]
        self.date = self.df_train.index.values
        self.n_rows = len(self.df_train)
        self.multiple_days_per_iter = multiple_days_per_iter

        assert self.n_rows > 0, 'no data to emulate'

    def predict(self, df: pd.DataFrame) -> None:
        # if you want to emulate public LB, store your prediction here and calculate MAE
        pass

    def iter_test(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        if self.multiple_days_per_iter:
            for i in range(self.n_rows // 2):
                date1 = self.date[2 * i]
                date2 = self.date[2 * i + 1]
                sample_sub1 = self._make_sample_sub(date1)
                sample_sub2 = self._make_sample_sub(date2)
                sample_sub = pd.concat([sample_sub1, sample_sub2]).reset_index(drop=True)
                df = self.df_train.loc[date1:date2]

                yield df, sample_sub.set_index('date')
        else:
            for i in range(self.n_rows):
                date = self.date[i]
                sample_sub = self._make_sample_sub(date)
                df = self.df_train.loc[date:date]

                yield df, sample_sub.set_index('date')

    def _make_sample_sub(self, date: int) -> pd.DataFrame:
        next_day = (pd.to_datetime(date, format='%Y%m%d') + pd.to_timedelta(1, 'd')).strftime('%Y%m%d')
        sample_sub = pd.DataFrame()
        sample_sub['date_playerId'] = next_day + '_' + self.players
        sample_sub['target1'] = 0
        sample_sub['target2'] = 0
        sample_sub['target3'] = 0
        sample_sub['target4'] = 0
        sample_sub['date'] = date
        return sample_sub


class MLBEmulator:
    def __init__(self,
                 data_dir: str = '../input/mlb-player-digital-engagement-forecasting',
                 eval_start_day: int = 20210401,
                 eval_end_day: Optional[int] = 20210430,
                 use_updated: bool = True,
                 multiple_days_per_iter: bool = False):
        self.data_dir = data_dir
        self.eval_start_day = eval_start_day
        self.eval_end_day = eval_end_day
        self.use_updated = use_updated
        self.multiple_days_per_iter = multiple_days_per_iter

    def make_env(self) -> Environment:
        return Environment(self.data_dir,
                           self.eval_start_day,
                           self.eval_end_day,
                           self.use_updated,
                           self.multiple_days_per_iter)


## How to use
All you need to do is to replace importing the `mlb` module with an instance of `MLBEmulator`.

In [2]:
emulation_mode = True

if emulation_mode:
    mlb = MLBEmulator(eval_start_day=20210701, eval_end_day=20210730)
else:
    import mlb

In [3]:
env = mlb.make_env()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: this is mock module for mlb
  from ipykernel import kernelapp as app


In [4]:
for n, (test_df, sample_prediction_df) in enumerate(env.iter_test()):
    display(test_df)
    display(sample_prediction_df)

,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
date,,,,,,,,,,,
20210701,"[{""engagementMetricsDate"":""2021-07-02"",""player...","[{""gamePk"":633405,""gameType"":""R"",""season"":2021...","[{""playerId"":448281,""gameDate"":""2021-07-01"",""t...","[{""home"":1,""gamePk"":633456,""gameDate"":""2021-07...","[{""home"":0,""teamId"":118,""gamePk"":633457,""gameD...","[{""transactionId"":500430,""playerId"":656342,""pl...","[{""season"":2021,""gameDate"":""2021-07-01"",""divis...",NaN,"[{""gamePk"":633456,""gameDate"":""2021-07-01"",""gam...","[{""date"":""2021-07-01"",""playerId"":506433,""playe...","[{""date"":""2021-07-01"",""teamId"":147,""teamName"":..."


,date_playerId,target1,target2,target3,target4
date,,,,,
20210701,20210702_593590,0,0,0,0
20210701,20210702_661269,0,0,0,0
20210701,20210702_669212,0,0,0,0
20210701,20210702_666201,0,0,0,0
20210701,20210702_680911,0,0,0,0
...,...,...,...,...,...
20210701,20210702_667674,0,0,0,0
20210701,20210702_672695,0,0,0,0
20210701,20210702_676103,0,0,0,0


,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
date,,,,,,,,,,,
20210702,"[{""engagementMetricsDate"":""2021-07-03"",""player...","[{""gamePk"":633462,""gameType"":""R"",""season"":2021...","[{""playerId"":453286,""gameDate"":""2021-07-02"",""t...","[{""home"":1,""gamePk"":633451,""gameDate"":""2021-07...","[{""home"":0,""teamId"":112,""gamePk"":633462,""gameD...","[{""transactionId"":500646,""playerId"":622100,""pl...","[{""season"":2021,""gameDate"":""2021-07-02"",""divis...",NaN,"[{""gamePk"":633470,""gameDate"":""2021-07-02"",""gam...",NaN,NaN


,date_playerId,target1,target2,target3,target4
date,,,,,
20210702,20210703_593590,0,0,0,0
20210702,20210703_661269,0,0,0,0
20210702,20210703_669212,0,0,0,0
20210702,20210703_666201,0,0,0,0
20210702,20210703_680911,0,0,0,0
...,...,...,...,...,...
20210702,20210703_667674,0,0,0,0
20210702,20210703_672695,0,0,0,0
20210702,20210703_676103,0,0,0,0


,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
date,,,,,,,,,,,
20210703,"[{""engagementMetricsDate"":""2021-07-04"",""player...","[{""gamePk"":633416,""gameType"":""R"",""season"":2021...","[{""playerId"":452657,""gameDate"":""2021-07-03"",""t...","[{""home"":1,""gamePk"":633440,""gameDate"":""2021-07...","[{""home"":0,""teamId"":158,""gamePk"":633387,""gameD...","[{""transactionId"":500865,""playerId"":668663,""pl...","[{""season"":2021,""gameDate"":""2021-07-03"",""divis...",NaN,"[{""gamePk"":633401,""gameDate"":""2021-07-03"",""gam...",NaN,NaN


...

,date_playerId,target1,target2,target3,target4
date,,,,,
20210715,20210716_593590,0,0,0,0
20210715,20210716_661269,0,0,0,0
20210715,20210716_669212,0,0,0,0
20210715,20210716_666201,0,0,0,0
20210715,20210716_680911,0,0,0,0
...,...,...,...,...,...
20210715,20210716_667674,0,0,0,0
20210715,20210716_672695,0,0,0,0
20210715,20210716_676103,0,0,0,0


,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
date,,,,,,,,,,,
20210716,"[{""engagementMetricsDate"":""2021-07-17"",""player...","[{""gamePk"":633283,""gameType"":""R"",""season"":2021...","[{""playerId"":448179,""gameDate"":""2021-07-16"",""t...","[{""home"":0,""gamePk"":633266,""gameDate"":""2021-07...","[{""home"":0,""teamId"":146,""gamePk"":633447,""gameD...","[{""transactionId"":505213,""playerId"":621043,""pl...","[{""season"":2021,""gameDate"":""2021-07-16"",""divis...",NaN,"[{""gamePk"":633346,""gameDate"":""2021-07-16"",""gam...",NaN,NaN


,date_playerId,target1,target2,target3,target4
date,,,,,
20210716,20210717_593590,0,0,0,0
20210716,20210717_661269,0,0,0,0
20210716,20210717_669212,0,0,0,0
20210716,20210717_666201,0,0,0,0
20210716,20210717_680911,0,0,0,0
...,...,...,...,...,...
20210716,20210717_667674,0,0,0,0
20210716,20210717_672695,0,0,0,0
20210716,20210717_676103,0,0,0,0


,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
date,,,,,,,,,,,
20210717,"[{""engagementMetricsDate"":""2021-07-18"",""player...","[{""gamePk"":633267,""gameType"":""R"",""season"":2021...","[{""playerId"":445926,""gameDate"":""2021-07-17"",""t...",NaN,NaN,"[{""transactionId"":505466,""playerId"":595928,""pl...","[{""season"":2021,""gameDate"":""2021-07-17"",""divis...",NaN,NaN,NaN,NaN


,date_playerId,target1,target2,target3,target4
date,,,,,
20210717,20210718_593590,0,0,0,0
20210717,20210718_661269,0,0,0,0
20210717,20210718_669212,0,0,0,0
20210717,20210718_666201,0,0,0,0
20210717,20210718_680911,0,0,0,0
...,...,...,...,...,...
20210717,20210718_667674,0,0,0,0
20210717,20210718_672695,0,0,0,0
20210717,20210718_676103,0,0,0,0
